In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import pyproj 

from shapely.geometry import Point
from shapely.geometry import LineString

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.linear_model import LogisticRegression

from tqdm import tqdm

#import xgboost as xgb
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier, plot_importance

import warnings
from warnings import simplefilter
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [5]:
class Config:
    PATH = '../../data/'

class Params:
    PARAM = 0

In [6]:
config = Config()
p = Params()

In [5]:
df = pd.read_csv(config.PATH+'E-tmja-2019-clean.csv')

In [6]:
columns_to_drop = ['dateReferentiel','zD','zF']
df = df.drop(columns=columns_to_drop)

In [7]:
import pyproj

def add_lat_lon_columns(df):
    """
    Adds new columns 'lonD', 'latD', 'lonF', 'latF' to the dataframe with
    corresponding latitude and longitude values based on the 'xD', 'yD', 'xF',
    and 'yF' columns, which are in Lambert-93 projection.

    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe containing the columns 'xD', 'yD', 'xF', and 'yF'.

    Returns:
    --------
    pandas.DataFrame
        The original dataframe with the new 'lonD', 'latD', 'lonF', and 'latF'
        columns added.
    """
    # Define the input and output projections
    in_proj = pyproj.Proj(init='epsg:2154')  # Lambert-93
    out_proj = pyproj.Proj(init='epsg:4326')  # WGS84

    df['xD'] = df['xD'].str.replace(',', '.')
    df['yD'] = df['yD'].str.replace(',', '.')
    df['xF'] = df['xF'].str.replace(',', '.')
    df['yF'] = df['yF'].str.replace(',', '.')

    # Convert start coordinates to lat-long
    df['lonD'], df['latD'] = pyproj.transform(in_proj, out_proj, df['xD'], df['yD'])

    # Convert end coordinates to lat-long
    df['lonF'], df['latF'] = pyproj.transform(in_proj, out_proj, df['xF'], df['yF'])

    return df

In [8]:
df = add_lat_lon_columns(df)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


In [310]:
df.head()

,route,longueur,prD,depPrD,concessionPrD,absD,cumulD,xD,yD,prF,...,yF,anneeMesureTrafic,typeComptageTrafic,typeComptageTrafic_lib,TMJA,ratio_PL,lonD,latD,lonF,latF
0,31D0044,44,0,31,N,0,0,511656.78,6204078.36,0,...,6204063.93,NaN,NaN,NaN,NaN,NaN,0.695459,42.911382,0.695975,42.911263
1,31D0044E,762,0,31,N,0,0,511367.22,6204210.06,0,...,6203732.4,NaN,NaN,NaN,NaN,NaN,0.691871,42.912490,0.698322,42.908331
2,69D0301,6055,2,69,N,-981,0,844036.84,6510806.45,7,...,6509342.34,NaN,NaN,NaN,NaN,NaN,4.850575,45.681629,4.918315,45.667307
3,69D0383,13752,4,69,N,-494,0,846831.3,6522372.16,17,...,6512429.8,NaN,NaN,NaN,NaN,NaN,4.890020,45.785151,4.844653,45.696348
4,69D0383BPNL,2408,0,69,N,0,0,844982.46,6522772.27,2,...,6522372.16,NaN,NaN,NaN,NaN,NaN,4.866352,45.789149,4.890020,45.785151


### Clean data

In [14]:
missing_values = df.isna().sum().sort_values(ascending=False)
missing_values[:6]

ratio_PL                  1634
TMJA                       954
typeComptageTrafic_lib     417
typeComptageTrafic         417
anneeMesureTrafic          138
route                        0
dtype: int64

In [11]:
import missingno as msno

# keep only columns with nan values
columns_with_nan = missing_values[missing_values.values!=0].index
msno.matrix(df[columns_with_nan])

<AxesSubplot:>

### Données stations


In [13]:
df_stations = pd.read_csv(config.PATH+'I-Données de stations TE_DV.xlsx - export_data_te.csv')

In [6]:
df_stations.head()

,URL,Station de service,Ville,Adresse,Coordinates,Energies,Lavage,Paiement,Produits & Services,Services,Club TotalEnergies,Truck,H2 Conversion
0,https://store.totalenergies.fr/fr_FR/NF059116,RELAIS SERIGUETTE,AIMARGUES,"2 bis. ROUTE DE LA PETITE CAMARGUE , 30470 AI...","43.6961378,4.2007447","AdBlue Truck, Excellium 98, Excellium Diesel, ...",NaN,"Routex, UTA, DKV, Mobility de TotalEnergies, M...","AdBlue Truck,","Café, Justbip, Wifi, AdBlue bidon,","Assistance Club 10 Jours, Jauge Cadeau, Cagnot...","Truck, Truck store,",0.0
1,https://store.totalenergies.fr/fr_FR/NF080036,RELAIS DEVEZE,BEZIERS,"1 AVENUE DE LA DEVEZE , 34500 BEZIERS, FRANCE","43.33150089999999,3.2474023","AdBlue Truck, Super Ethanol e85, Excellium 98,...","Lavage Haute Pression, Aspirateur, Lavage à Ro...","UTA, Carte Jubileo, MOL, Carte Wash, American ...","AdBlue Truck,","FDJ, Café, Lavage à Rouleaux, Justbip, Gonflag...","Assistance Club 10 Jours, Jauge Cadeau, Cagnot...","Truck, Truck store,",0.0
2,https://store.totalenergies.fr/fr_FR/NF080198,RELAIS SAINT MATHURIN,ALLONNE,"ZAC SAINT MATHURIN - RD 1001 , 60000 ALLONNE,...","49.409445,2.12815","Borne DC175 kW, AdBlue Truck, Excellium 98, Ex...","Aspirateur, Lavage à Rouleaux,","Visa, American Express, Mastercard, Eurotrafic...","AdBlue Truck,","Café, Lavage à Rouleaux, Gonflage, Wifi, Justb...","Assistance Club 10 Jours, Jauge Cadeau, Cagnot...","Truck, Pompes PL,",1.0
3,https://store.totalenergies.fr/fr_FR/NF080197,REL.LORRAINE LES RAPPES,SANDAUCOURT,"AUT.A31-LORRAINE LES RAPPES , 88170 SANDAUCOU...","48.2659392,5.863884","Borne AC43/DC50 kW, Borne DC175 kW, SP95 e10, ...",NaN,"Mobility de TotalEnergies, Carte Jubileo, Cart...","AdBlue Truck,","Défibrillateur, Café, Wifi, Autoroutière, Toil...","Assistance Club 30 Jours, Club Truck, Cagnotte...","Truck, Pompes PL,",0.0
4,https://store.totalenergies.fr/fr_FR/NF058916,REL.MORAINVILLIERS NORD,MORAINVILLIERS,"A13 - AIRE DE MORAINVILLIERS NORD , 78630 MOR...","48.9411429,1.9543387","Diesel, GPL, SP95 e10, Excellium 98, GNR, Exce...",NaN,"Mobility de TotalEnergies, MOL, American Expre...","AdBlue Truck,","Click & Collect Lyf, Toilettes, Défibrillateur...","Assistance Club 30 Jours, Club Truck, Cagnotte...","Truck, Parking PL, Truck store,",0.0


In [26]:
def create_coordinate_columns(
        df_stations: pd.DataFrame
):
    #create Latitude and Longitude coordinates 
    df_stations['Latitude'] = df_stations['Coordinates'].apply(lambda x: x.split(',')[0].strip())
    df_stations['Longitude'] = df_stations['Coordinates'].apply(lambda x: x.split(',')[1].strip())

    #filter out the empty coordinates
    df_stations = df_stations[(df_stations['Longitude'] != '')&(df_stations['Latitude'] != '')]

    return df_stations

In [27]:
df_stations = create_coordinate_columns(df_stations)

### Geopandas


In [7]:
df_shp = gpd.read_file(config.PATH+'E-tmja2019-shp/TMJA2019.shp')

In [8]:
df_shp.shape[0]

4695

In [9]:
df_shp[df_shp['geometry'].geom_type == 'LineString'].shape[0]

4629

There are 66 multistring.

In [11]:
df_shp = df_shp.set_index(['route'])

In [33]:
def transform_station_coordinates(
        df: pd.DataFrame
        )-> pd.DataFrame:
    """
    Transform the geometry column Linestrings currently in Lambert-93 projection
    into the corresponding latitude and longitude.

    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe containing the geopandas geometry column

    Returns:
    --------
    pandas.DataFrame
        The original dataframe with the new geometry column.
    """
    # Define the input and output projections
    in_proj = pyproj.Proj(init='epsg:2154')  # Lambert-93
    out_proj = pyproj.Proj(init='epsg:4326')  # WGS84

    transform_coord = lambda x: pyproj.transform(out_proj, in_proj, float(x.split(',')[0]),float(x.split(',')[1]))

    coord_transform = []
    for x in df['Coordinates'].values:
        coord_transform.append(transform_coord(x))
    
    df['Coordinate_transform'] = coord_transform 

    return df

In [ ]:
df_stations_h2 = df_stations[df_stations['H2 Conversion']==1]

In [34]:
# transform the station coordinates in the correct form 
df_stations_trans = transform_station_coordinates(df_stations)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


KeyboardInterrupt: 

In [ ]:
df_stations_trans.head()

,URL,Station de service,Ville,Adresse,Coordinates,Energies,Lavage,Paiement,Produits & Services,Services,Club TotalEnergies,Truck,H2 Conversion,Latitude,Longitude,Coordinate_transform
2,https://store.totalenergies.fr/fr_FR/NF080198,RELAIS SAINT MATHURIN,ALLONNE,"ZAC SAINT MATHURIN - RD 1001 , 60000 ALLONNE,...","49.409445,2.12815","Borne DC175 kW, AdBlue Truck, Excellium 98, Ex...","Aspirateur, Lavage à Rouleaux,","Visa, American Express, Mastercard, Eurotrafic...","AdBlue Truck,","Café, Lavage à Rouleaux, Gonflage, Wifi, Justb...","Assistance Club 10 Jours, Jauge Cadeau, Cagnot...","Truck, Pompes PL,",1.0,49.409445,2.12815,"(7045319.394544157, 3132226.9412886)"
6,https://store.totalenergies.fr/fr_FR/NF080155,REL.LIMOURS JANVRY,JANVRY,"A10 AIRE DE LIMOURS JANVRY , 91640 JANVRY, FR...","48.6371317,2.1488045","Borne DC175 kW, AdBlue Truck, Excellium 98, Ex...","Aspirateur, Lavage à Rouleaux,","American Express, DKV, Mastercard, UTA, Visa, ...","AdBlue Truck,","Défibrillateur, Autoroutière, Lavage à Rouleau...","Assistance Club 30 Jours, Club Truck, Cagnotte...","Truck, Parking PL, Truck store,",1.0,48.6371317,2.1488045,"(6950246.0744015165, 3073113.203610874)"
11,https://store.totalenergies.fr/fr_FR/NF030340,POREZ AUTO LOCA STATION,ST QUENTIN,"98 RUE ALEXANDRE DUMAS , 02100 ST QUENTIN, FR...","49.8486264,3.2713774","Excellium Grand Froid, AdBlue Truck, Excellium...",NaN,"Routex, Mobility de TotalEnergies, American Ex...","AdBlue Truck,","Baie de maintenance, Boutique, Wifi,","Assistance Club 10 Jours, Jauge Cadeau, Cagnot...","Truck,",1.0,49.8486264,3.2713774,"(7006730.337569671, 3303291.312808282)"
14,https://store.totalenergies.fr/fr_FR/NF040392,STATION SERVICE MEDINE,VALLETOT,"33 RTE DE MEDINE - CD.89 , 27350 VALLETOT, FR...","49.3722338,0.6275373","AdBlue Truck, Excellium 98, Excellium Diesel, ...",NaN,"American Express, Mastercard, Eurotrafic, Visa...","AdBlue Truck,","Boutique, Wifi, AdBlue bidon,","Assistance Club 30 Jours, Club Truck, Cagnotte...","Truck,",1.0,49.3722338,0.6275373,"(7161702.210099935, 2947642.230104704)"
16,https://store.totalenergies.fr/fr_FR/NF078008,RELAIS GRAND PRE,GOUSSAINVILLE,"32/38 AV DE LA GARE , 95190 GOUSSAINVILLE, FR...","49.0155943,2.4703296","Excellium 98, Excellium Diesel, Diesel, Super ...","Aspirateur, Lavage à Rouleaux,","DKV, UTA, MOL, Visa, Carte Jubileo, Carte Club...",NaN,"Wifi, Boutique, Baie de maintenance, Lavage à ...","Assistance Club 30 Jours, Club Truck, Cagnotte...","Truck,",1.0,49.0155943,2.4703296,"(6970667.674578585, 3141769.8504718314)"


In [45]:
def transform_coordinates(df_shp:pd.DataFrame,
                     nb_decimal: int = 1):
    
    # select only routes with LineString geometries
    df_line = df_shp[df_shp['geometry'].geom_type == 'LineString']

    in_proj = pyproj.Proj(init='epsg:2154')  # Lambert-93
    out_proj = pyproj.Proj(init='epsg:4326')  # WGS84
    transform_coords = lambda x: pyproj.transform(in_proj, out_proj, float(x[0]),float(x[1]))

    list_geo = []
    for x in tqdm(df_line["geometry"]):
        list_geo.append([(round(transform_coords(c)[1],3),round(transform_coords(c)[0],3)) for c in x.coords[:]])

    # apply the lambda function to the geometry column of the Geodataframe
    df_line["geometry_lat_long"] = list_geo

    return df_line

In [78]:
df_line_trans = transform_coordinates(df_shp)

  0%|          | 0/4629 [00:00<?, ?it/s]/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  if __name__ == '__main__':
 26%|██▌       | 1193/4629 [4:59:03<21:43:48, 22.77s/it]   

In [46]:
def round_coordinates(df_shp:pd.DataFrame,
                     nb_decimal: int = 1):
    
    # select only routes with LineString geometries
    df_line = df_shp[df_shp['geometry'].geom_type == 'LineString']

    list_geo = []
    for x in tqdm(df_line["geometry"]):
        list_geo.append([(round(float(c[0])/100), round(float(c[1])/100)) for c in x.coords[:]])

    # apply the lambda function to the geometry column of the Geodataframe
    df_line["geometry_round"] = list_geo

    return df_line

In [47]:
df_line = round_coordinates(df_shp)

100%|██████████| 4629/4629 [00:02<00:00, 1571.21it/s]
/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [58]:
df_line.head()

,dateRefere,longueur,prD,depPrD,concession,absD,cumulD,xD,yD,zD,...,xF,yF,zF,anneeMesur,typeCompta,typeComp_1,TMJA,ratio_PL,geometry,geometry_round
route,,,,,,,,,,,,,,,,,,,,,
31D0044,2019-01-01,44.0,0,31,N,0.0,0.0,511656.78,6204078.36,0.0,...,511698.51,6204063.93,0.0,0,None,None,0,0.0,"LINESTRING (511656.785 6204078.361, 511672.731...","[(5117, 62041), (5117, 62041), (5117, 62041), ..."
69D0301,2019-01-01,6055.0,2,69,N,-981.0,0.0,844036.84,6510806.45,0.0,...,849346.89,6509342.34,0.0,0,None,None,0,0.0,"LINESTRING (844036.841 6510806.450, 844044.447...","[(8440, 65108), (8440, 65108), (8440, 65108), ..."
69D0383,2019-01-01,13752.0,4,69,N,-494.0,0.0,846831.30,6522372.16,0.0,...,843537.87,6512429.80,0.0,0,None,None,0,0.0,"LINESTRING (846831.303 6522372.160, 846865.124...","[(8468, 65224), (8469, 65223), (8469, 65223), ..."
69D0383BPNL,2019-01-01,2408.0,0,69,N,0.0,0.0,844982.46,6522772.27,0.0,...,846831.30,6522372.16,0.0,0,None,None,0,0.0,"LINESTRING (844982.456 6522772.273, 844995.417...","[(8450, 65228), (8450, 65228), (8450, 65228), ..."
75000C01,2019-01-01,35040.0,0,75,N,0.0,0.0,655221.44,6858722.80,0.0,...,655221.44,6858722.80,0.0,0,None,None,0,0.0,"LINESTRING (647256.052 6864493.521, 647272.326...","[(6473, 68645), (6473, 68645), (6473, 68646), ..."


In [49]:
df_line.to_csv(config.PATH+'routes_round_coord.csv')

In [74]:
def intersection_road_station(
        df_line: pd.DataFrame, 
        station_coordinates: str, 
        route_index: int,
        nb_decimal: int = 1
        ) -> str:
    '''
    Round the coordinates and return the name of the route if the station is contained in the route
    '''

    # round the station points 
    #point = Point(
    point = (
        round(station_coordinates[0]/100),
        round(station_coordinates[1]/100))
    #point = (
        #round(float(station_coordinates.split(',')[0]),nb_decimal),
       # round(float(station_coordinates.split(',')[1]),nb_decimal))

    line = df_line['geometry_round'].iloc[route_index]

    # Check if the station point is on the route LineString
    #is_on_line = line.intersects(point)
    #EPS = 1
    #is_on_line = line.buffer(EPS).contains(point)
    is_on_line = point in line
    
    if is_on_line:
        return df_line.index[route_index]
    else:
        return ''

In [75]:
def create_list_routes(
        station_coord: tuple,
        df_line: pd.DataFrame
):
        list_routes =  [intersection_road_station(df_line,station_coord,route) for route in range(df_line.shape[0])]
        return list_routes

In [76]:
#test function for one station
station_coord = df_stations_h2_trans['Coordinate_transform'].iloc[0]
results = create_list_routes(station_coord,df_line)

In [77]:
''.join(results)

''

In [69]:
df_stations_h2_trans['Coordinate_transform_round'] = df_stations_h2_trans['Coordinate_transform'].apply(lambda x: (round(x[0]/10),round(x[1]/100)))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
df_stations_h2_trans['Coordinate_transform_round'].iloc[:10]

2     (70453, 31322)
6     (69502, 30731)
11    (70067, 33033)
14    (71617, 29476)
16    (69707, 31418)
17    (68824, 24682)
18    (61999, 29903)
20    (69090, 31293)
28    (65443, 32661)
29    (64055, 29952)
Name: Coordinate_transform_round, dtype: object

In [70]:
(7045, 31322) in df_line['geometry_round']

False

In [63]:
df_line['geometry_round']

route
31D0044        [(5117, 62041), (5117, 62041), (5117, 62041), ...
69D0301        [(8440, 65108), (8440, 65108), (8440, 65108), ...
69D0383        [(8468, 65224), (8469, 65223), (8469, 65223), ...
69D0383BPNL    [(8450, 65228), (8450, 65228), (8450, 65228), ...
75000C01       [(6473, 68645), (6473, 68645), (6473, 68646), ...
                                     ...                        
N1338          [(5601, 69279), (5601, 69279), (5601, 69279), ...
P0224          [(5560, 62889), (5560, 62889), (5560, 62889), ...
P0524          [(4403, 63611), (4403, 63610), (4403, 63610), ...
P0524          [(4588, 63259), (4588, 63259), (4588, 63259), ...
P0542          [(5487, 62837), (5487, 62837), (5487, 62837), ...
Name: geometry_round, Length: 4629, dtype: object

In [56]:
''.join(results)

''

In [392]:
def create_col_routes(
        df_stations_t: pd.DataFrame,
        df_line: pd.DataFrame
):
    '''
    Function creates the column Routes_on with a list of 
    '''
    # Crate the column Routes_on with a list of the route the station is on
    df_stations_t['Routes_on'] = df_stations_t['Coordinate_transform'].apply(lambda station_coord: create_list_routes(station_coord,df_line))
    # Join the list to check if there are any non-empty routes
    df_stations_t['Count_routes_on'] = df_stations_t['Routes_on'].apply(lambda my_list: ''.join(my_list))

    return df_stations_t

In [393]:
df_stations_routes = create_col_routes(df_stations_t[:10], df_line)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [394]:
df_stations_routes[df_stations_routes['Count_routes_on']!='']

,URL,Station de service,Ville,Adresse,Coordinates,Energies,Lavage,Paiement,Produits & Services,Services,Club TotalEnergies,Truck,H2 Conversion,Latitude,Longitude,Routes_on,Count_routes_on,Coordinate_transform


In [348]:
#df_stations_t.to_csv(config.PATH+'stations_transformed_coordinates.csv')

### Donnees hub

In [ ]:
df_hub = gpd.read_file(config.PATH+'Aires_logistiques_denses.shp')

In [ ]:
df_hub

In [ ]:
df_hub_large = gpd.read_file(config.PATH+'Aires_logistiques_denses.shp')
df_hub_large